# Initialising LUSID

In [1]:
# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import os
import helper_functions as fun

# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

pd.set_option("display.precision", 8)
pd.set_option("display.float_format", "{:0.4f}".format)

print("LUSID Environment Initialised")
print(
    "LUSID API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version:  0.0.1.0


In [2]:
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

![Scopes](img/paper-lusid.gif)

## 1) Creating a logically separated environment for each analyst

In [3]:
# Fetch our scopes
analyst_scope_code = fun.create_analyst_scope()

Analyst Scope Code: analyst-paper-3b22-f8c5-e45f-72


*Our LUSID environment now looks like the below. We will update this with each step.*

![Scopes](img/paper-scopes.gif)

## 2) Creating our tradeable instrument universe for each analyst

In [4]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv("data/paper-instruments.csv")
# Look at the first 10 instruments
instrument_universe.sort_values(by="instrument_name").head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,client_internal
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,imd_56344535
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,263494,imd_56344983
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,imd_56344098
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,imd_56344345
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,imd_56344363
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,imd_56344296
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,94627,imd_56344187
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,imd_56344572
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,207458,imd_56344789
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,880882,imd_56344132


In [5]:
batch_upsert_request = fun.batch_upsert(instrument_universe, api_factory)

Instruments Successfully Upserted: 


In [6]:
# Call LUSID to upsert our batch
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body=batch_upsert_request
)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier="Figi").sort_values(
    by="Instrument"
).head(10)

Instruments Successfully Upserted: 


,Instrument,Figi ID,LUSID Instrument ID
10,AVEVA GRP,BBG000C21Y87,LUID_00003D5F
16,BAE SYS.,BBG000BD5TW4,LUID_00003D5L
0,BATM ADVANCED,BBG000BFJD77,LUID_00003D55
7,BTG,BBG000BZZK79,LUID_00003D5C
12,COBHAM,BBG000BS6810,LUID_00003D5H
27,COMPUTACENTER,BBG000BN7CL9,LUID_00003D5W
17,CONSORT MED.,BBG000BD8278,LUID_00003D5M
11,DIALIGHT,BBG000BLQNG4,LUID_00003D5G
2,GENUS,BBG000CTMKX7,LUID_00003D57
28,GRESHAM TECH,BBG000BS2592,LUID_00003D5X


Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/paper-instrumentmaster.gif)

## 3) Creating a transactions portfolio

A transactions portfolio is initalised to hold all transactions made by an analyst. 

A second portfolio is also initalised which will store the index of the instrument universe to be used as a benchmark.

In [7]:
# Define unique codes for each of our portfolios
transaction_portfolio_code = "LSE_FTSE_techMarkFocus_transactions_DEMO"
reference_portfolio_code = "LSE_FTSE_techMarkFocus_reference_DEMO"
# Pretty print our codes
prettyprint.heading("Transaction Portfolio Code", transaction_portfolio_code)
prettyprint.heading("Reference Portfolio Code", reference_portfolio_code)

# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3)).isoformat()

descriptions = "Paper transaction portfolio_DEMO"
portfolio_codes = [transaction_portfolio_code, reference_portfolio_code]


fun.request_transaction_portfolio_creation(
    transaction_portfolio_code,
    portfolio_creation_date,
    analyst_scope_code,
    descriptions,
    api_factory,
)

portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3)).isoformat()

fun.request_reference_portfolio_creation(
    reference_portfolio_code, portfolio_creation_date, analyst_scope_code, api_factory
)

Transaction Portfolio Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Reference Portfolio Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Created
Scope: analyst-paper-3b22-f8c5-e45f-72
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Portfolio Effective From: 2022-09-27 14:19:09.680151+00:00
Portfolio Created On: 2022-09-30 14:19:09.748348+00:00

Portfolio Created
Scope: analyst-paper-3b22-f8c5-e45f-72
Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Effective From: 2022-09-27 14:19:09.777331+00:00
Portfolio Created On: 2022-09-30 14:19:10.097322+00:00



![Scopes](img/paper-referenceportfolio.gif)

## 4) Populating our analyst's portfolio with a starting cash balance

In [8]:
# Set the date from which the cash balance will apply to be just after portfolio creation
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2.9)).isoformat()
# Define our initial cash balance
initial_cash_balance = 30000000

# # Create a holding adjustment to set our initial cash balance
fun.populate_with_cash(
    holdings_effective_date,
    initial_cash_balance,
    analyst_scope_code,
    transaction_portfolio_code,
    api_factory,
)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-3b22-f8c5-e45f-72
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Holdings Effective From: 2022-09-27 14:19:09.680151+00:00
Holdings Created On: 2022-09-30 14:19:10.425357+00:00



Our transactions portfolio is now populated with an initial cash balance and is no longer empty.

![Scopes](img/paper-transactionsportfoliofilled.gif)

## 5) Populating our reference portfolio 

The initaised reference portfolio is populated with correct weights such that it tracks the market as closely as possible.

In [9]:
# Import the market capitalisation of each of our instruments in the index
instrument_market_cap = pd.read_csv("data/paper-weights.csv")
# Take a look at the first 10 market capitalisations
instrument_market_cap.sort_values(by="instrument_name").head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,marketcap
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,4502.6800
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,0263494,16349.6600
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,185.2500
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,3202.2000
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,2579.9000
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,1161.9600
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,0094627,422.5600
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,133.3900
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,0207458,1455.7200
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,0880882,64.3400


In [10]:
# upsert contituents to LUSID
fun.upsert_constituents(
    instrument_market_cap,
    holdings_effective_date,
    analyst_scope_code,
    reference_portfolio_code,
    api_factory,
)

Constituents Upserted


![Scopes](img/paper-referenceportfoliofilled.gif)

## 6) Allow our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

In [11]:
# Import our analyst's trades from a CSV file
analyst_transactions = import_data.fetch_client_transactions(
    "paper-transactions.csv", days_back=2
)
analyst_transactions.sort_values(by="transaction_id").head(n=10)

,transaction_id,type,portfolio,instrument_name,instrument_uid,transaction_date,settlement_date,units,transaction_price,transaction_currency,total_cost,strategy,description
0,tid_124398219481,Buy,LSE_FTSE_techMarkFocus_transactions,AVEVA GRP,BBG000C21Y87,2022-09-28T10:17:26.753643+00:00,2022-09-30T10:17:26.753643+00:00,265600,28.9400,GBP,7686464.0000,quantitativeSignal,Purchase
4,tid_223239424244,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2022-09-28T16:57:36.301943+00:00,2022-09-30T16:57:36.301943+00:00,750000,0.0001,GBP,69.4500,fundamentalAnalysis,Brokerage Fees
9,tid_239394829484,Buy,LSE_FTSE_techMarkFocus_transactions,DIALIGHT,BBG000BLQNG4,2022-09-29T08:43:15.511733+00:00,2022-10-01T08:43:15.511733+00:00,1500000,4.1800,GBP,6270000.0000,quantitativeSignal,Purchase
17,tid_324599699055,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2022-09-29T15:26:56.881549+00:00,2022-10-01T15:26:56.881549+00:00,234000,0.0020,GBP,458.6400,quantitativeSignal,Stamp Duty
15,tid_334250655566,Buy,LSE_FTSE_techMarkFocus_transactions,MICROGEN,BBG000BDSS88,2022-09-28T11:11:52.551768+00:00,2022-09-30T11:11:52.551768+00:00,234000,3.9200,GBP,917280.0000,quantitativeSignal,Purchase
1,tid_339423984894,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2022-09-29T11:42:55.020604+00:00,2022-10-01T11:42:55.020604+00:00,265600,0.0058,GBP,1537.2928,quantitativeSignal,Brokerage Fees
2,tid_339423984896,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2022-09-28T08:07:31.718242+00:00,2022-09-30T08:07:31.718242+00:00,265600,0.0145,GBP,3843.2320,quantitativeSignal,Stamp Duty
14,tid_342398598555,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2022-09-28T10:21:49.306835+00:00,2022-09-30T10:21:49.306835+00:00,345000,0.0110,GBP,3795.0000,technicalAnalysis,Stamp Duty
6,tid_342398989895,SellShort,LSE_FTSE_techMarkFocus_transactions,COMPUTACENTER,BBG000BN7CL9,2022-09-28T08:51:29.563321+00:00,2022-09-30T08:51:29.563321+00:00,150000,10.5000,GBP,1575000.0000,fundamentalAnalysis,Purchase
20,tid_342453598349,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2022-09-28T08:42:45.364857+00:00,2022-09-30T08:42:45.364857+00:00,543000,0.0068,GBP,3692.4000,technicalAnalysis,Stamp Duty


Custom properties can be defined in LUSID. In this example, a trading strategy is specified created as a **property**, this allows grouping of transactions for the purpose of measuring performance, valuation etc...

In [12]:
# Create a request to define our strategy property
strategy_property_key = fun.request_define_property(
    domain="Transaction",
    scope=analyst_scope_code,
    code="strategy",
    display_name="strategy",
    api_factory=api_factory,
)

Strategy Property Key: : Transaction/analyst-paper-3b22-f8c5-e45f-72/strategy


In [13]:
# upsert all analyist transactions to LUSID
fun.upsert_trades(
    analyst_transactions,
    strategy_property_key,
    analyst_scope_code,
    transaction_portfolio_code,
    api_factory,
)

Transactions Successfully Upserted into Portfolio
Scope: analyst-paper-3b22-f8c5-e45f-72
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Transactions Effective From: 2022-09-29 16:00:44.780863+00:00
Transactions Created On: 2022-09-30 14:19:11.087860+00:00



Our transaction portfolio is now populated with our analyst's paper trades.

## 7) Upload market data quotes to hold the latest prices of our tradeable instrument universe

In [14]:
# Import our instrument prices from a CSV file
instrument_prices = pd.read_csv("data/paper-analytics.csv")
# Pretty print our pricing
instrument_prices.sort_values(by="instrument_name").head(n=10)

,instrument_name,currency,figi,price_original,price_current,ticker,isin,sedol
0,AVEVA GRP,GBP,BBG000C21Y87,27.8400,30.3178,AVV,GB00BBG9VN75,BBG9VN7
1,BAE SYS.,GBP,BBG000BD5TW4,5.1540,4.7829,BA.,GB0002634946,263494
2,BATM ADVANCED,GBP,BBG000BFJD77,0.4590,0.4246,BVC,IL0010849045,911146
3,BTG,GBP,BBG000BZZK79,8.2750,8.9701,BTG,GB0001001592,100159
4,COBHAM,GBP,BBG000BS6810,1.0715,1.0222,COB,GB00B07KD360,B07KD36
5,COMPUTACENTER,GBP,BBG000BN7CL9,10.6800,11.4596,CCC,GB00BV9FP302,BV9FP30
6,CONSORT MED.,GBP,BBG000BD8278,8.7000,9.3090,CSRT,GB0000946276,94627
7,DIALIGHT,GBP,BBG000BLQNG4,3.9200,3.5515,DIA,GB0033057794,3305779
8,GENUS,GBP,BBG000CTMKX7,22.4600,20.5958,GNS,GB0002074580,207458
9,GRESHAM TECH,GBP,BBG000BS2592,0.9450,1.0008,GHT,GB0008808825,880882


In [15]:
# Set our quote effective dates
quotes_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)
today = datetime.now(pytz.UTC)

fun.create_instrument_quotes(
    quotes_effective_date, today, instrument_prices, analyst_scope_code, api_factory
)

![Scopes](img/paper-quotestore.gif)

## 8) Value each portfolio using our market data quotes

In [16]:
recipe_scope = "sandbox"
recipe_code = "market_value"

# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.LusidInstrumentId.*",
                supplier="DataScope",
                data_scope=analyst_scope_code,
                quote_type="Price",
                field="Mid",
                quote_interval=quotes_effective_date.strftime("%Y-%m-%d"),
            )
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="DataScope",
            credit="DataScope",
            equity="DataScope",
            fx="DataScope",
            rates="DataScope",
        ),
        options=models.MarketOptions(
            default_supplier="DataScope",
            default_instrument_code_type="LusidInstrumentId",
            default_scope=analyst_scope_code,
        ),
    ),
)

# Upsert the recipe
upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id=models.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        models.AggregateSpec(key="Instrument/default/LusidInstrumentId", op="Value"),
        models.AggregateSpec(key="Instrument/default/Name", op="Value"),
        models.AggregateSpec(key="Holding/default/Units", op="Sum"),
        models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
        models.AggregateSpec(key="Valuation/PvInReportCcy", op="Sum"),
        models.AggregateSpec(key="Holding/default/Price", op="Sum"),
    ],
    group_by=["Instrument/default/LusidInstrumentId"],
    portfolio_entity_ids=[
        models.PortfolioEntityId(
            scope=analyst_scope_code, code=transaction_portfolio_code
        )
    ],
    valuation_schedule=models.ValuationSchedule(effective_at=today.isoformat()),
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request
)

df = pd.DataFrame(valuation.data)
df.loc["Total"] = df.sum(numeric_only=True)
df["return"] = (df["Sum(Valuation/PvInReportCcy)"] - df["Sum(Holding/default/Cost)"]) / df[
    "Sum(Holding/default/Cost)"
]
df

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Valuation/PvInReportCcy),Sum(Holding/default/Price),return
0,CCY_GBP,GBP,4364776.0000,4364776.0000,4364776.0000,20.0000,0.0000
1,LUID_00003D5W,COMPUTACENTER,-150000.0000,-1575000.0000,-1718946.0000,11.4596,0.0914
2,LUID_00003D5Z,ULTRA ELEC.,120000.0000,1582800.0000,1603546.8000,13.3629,0.0131
3,LUID_00003D5F,AVEVA GRP,265600.0000,7686464.0000,8052397.0560,30.3178,0.0476
4,LUID_00003D5D,MICROGEN,234000.0000,917280.0000,843094.9800,3.6030,-0.0809
5,LUID_00003D5V,SAGE GRP.,-98000.0000,-577220.0000,-643432.7200,6.5656,0.1147
6,LUID_00003D55,BATM ADVANCED,750000.0000,347250.0000,318431.2500,0.4246,-0.0830
7,LUID_00003D5G,DIALIGHT,1500000.0000,6270000.0000,5327280.0000,3.5515,-0.1504
8,LUID_00003D56,XP POWER,98000.0000,2156000.0000,1912019.2000,19.5104,-0.1132
9,LUID_00003D5A,MICRO FOCUS,-543000.0000,-7384800.0000,-7503461.7900,13.8185,0.0161


## 9) Compare performance across the analyst's reference and transaction portfolios

In [17]:
# create index portfolio
index_portfolio_code = "LSE_FTSE_techMarkFocus_index_DEMO"
prettyprint.heading("Index Portfolio Code", index_portfolio_code)

description = "Index transaction portfolio_DEMO"
fun.request_transaction_portfolio_creation(
    index_portfolio_code,
    portfolio_creation_date,
    analyst_scope_code,
    description,
    api_factory,
)

Index Portfolio Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Created
Scope: analyst-paper-3b22-f8c5-e45f-72
Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Effective From: 2022-09-27 14:19:09.777331+00:00
Portfolio Created On: 2022-09-30 14:19:12.494640+00:00



We now have an index portfolio that we can use to hold our index and obtain our benchmark.

![Scopes](img/paper-indexportfolio.gif)

In [18]:
# setup an index portfolio
index_setup = fun.setup_index(
    analyst_scope_code, reference_portfolio_code, instrument_prices, api_factory
)

# Call LUSID to set all of our holdings to the initial index level
adjust_holdings_response = api_factory.build(
    lusid.api.TransactionPortfoliosApi
).set_holdings(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    effective_at=portfolio_creation_date,
    adjust_holding_request=index_setup,
)

prettyprint.set_holdings_response(
    adjust_holdings_response, analyst_scope_code, index_portfolio_code
)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-3b22-f8c5-e45f-72
Code: LSE_FTSE_techMarkFocus_index_DEMO
Holdings Effective From: 2022-09-27 14:19:09.777331+00:00
Holdings Created On: 2022-09-30 14:19:13.595797+00:00



We now have an index portfolio that we can value.

![Scopes](img/paper-indexportfoliofilled.gif)

In [19]:
# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id=models.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        models.AggregateSpec(key="Instrument/default/LusidInstrumentId", op="Value"),
        models.AggregateSpec(key="Instrument/default/Name", op="Value"),
        models.AggregateSpec(key="Holding/default/Units", op="Sum"),
        models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
        models.AggregateSpec(key="Valuation/PvInReportCcy", op="Sum"),
        models.AggregateSpec(key="Holding/default/Price", op="Sum"),
    ],
    group_by=["Instrument/default/LusidInstrumentId"],
    portfolio_entity_ids=[
        models.PortfolioEntityId(scope=analyst_scope_code, code=index_portfolio_code)
    ],
    valuation_schedule=models.ValuationSchedule(effective_at=today.isoformat()),
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request
)

df = pd.DataFrame(valuation.data)
df.loc["Total"] = df.sum(numeric_only=True)
df["return"] = (df["Sum(Valuation/PvInReportCcy)"] - df["Sum(Holding/default/Cost)"]) / df[
    "Sum(Holding/default/Cost)"
]
df

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Valuation/PvInReportCcy),Sum(Holding/default/Price),return
0,LUID_00003D5F,AVEVA GRP,2.0777,57.8400,62.9924,30.3178,0.0891
1,LUID_00003D5L,BAE SYS.,40.7525,210.0400,194.9154,4.7829,-0.0720
2,LUID_00003D55,BATM ADVANCED,5.1848,2.3800,2.2014,0.4246,-0.0751
3,LUID_00003D5C,BTG,4.9713,41.1400,44.5931,8.9701,0.0839
4,LUID_00003D5H,COBHAM,30.9314,33.1400,31.6185,1.0222,-0.0459
5,LUID_00003D5W,COMPUTACENTER,1.3977,14.9300,16.0170,11.4596,0.0728
6,LUID_00003D5M,CONSORT MED.,0.6240,5.4300,5.8085,9.3090,0.0697
7,LUID_00003D5G,DIALIGHT,0.4371,1.7100,1.5525,3.5515,-0.0921
8,LUID_00003D57,GENUS,0.8326,18.7000,17.1489,20.5958,-0.0829
9,LUID_00003D5X,GRESHAM TECH,0.8747,0.8300,0.8753,1.0008,0.0546
